<h1 style='background-color: BLACK; padding: 10px; color: white'> Zoopla </h1>

<h1 style='background-color: BLACK; padding: 10px; color: white'> Sales </h1>


First things first, we will import the required libraries

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

Next, we will load in the dataset containing the list of postcode for this project

In [5]:
codes = pd.read_csv('London postcode districts.xlsx - PC DIST.csv')

Now, we will create a function called 'zoopla_sales' to scrap properties that are for sale on the Zoopla website

In [18]:
def zoopla_sales(postcodes, Trans_type, website, df):
    """
    This function scrapes property data from Rightmove for a given list of postcodes and transaction type,
    and returns the data as a pandas DataFrame.

    Arguments:
    postcodes -- a list of postcodes for which to scrape property data
    Trans_type -- the transaction type of the properties to be scraped ('sales' or 'rent')
    website -- the name of the website being scraped (in this case, 'Rightmove')
    df -- an empty pandas DataFrame to store the scraped data

    Returns:
    df1 -- a pandas DataFrame containing the scraped property data
    
    """    
    # Set the path to the ChromeDriver executable
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    
    # Launch the ChromeDriver with the specified service
    driver = webdriver.Chrome(service= s)
    
    # Navigate to the specified URL 
    driver.get('https://www.zoopla.co.uk/') 

    # Click to cancel the pop-up window and maximize the window
    #driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div[2]/button').click()
    time.sleep(2)
    driver.maximize_window()

    # Find the search bar  
    time.sleep(1.2)
    search = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[1]/div/div/div/div/div/div/input')
    search.send_keys(postcode)
    time.sleep(1.3)
    
    
    # Click the search button
    time.sleep(3)
    driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[2]/button').click()
    time.sleep(40)
    
    # Click the human verification botton
    try:
        driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/div/div[1]/table/tbody/tr/td[1]/div[1]').click()
        driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/div/div[1]/table/tbody/tr/td[1]/div[1]/div/label/input').click()
        
    
    except:
        print('I no see anything bros')
    
    
    # Initialize empty lists for storing scraped data
    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    # Create an empty dataframe to store the scraped data
    df = pd.DataFrame()
    
     # Setting the page number to be 1 
    i = 1
    while True:
        # Scrapping data for the required features in the first page
        time.sleep(2)
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        address_list = driver.find_elements(By.XPATH, '//div[1]/div/div/div/div[2]/div/a/div/div[3]/h3') 
        type_list = driver.find_elements(By.XPATH, "//div[1]/div/div/div/div[2]/div/a/div/div[3]/h2") 
        bedroom_list = driver.find_elements(By.XPATH, '//div[1]/div/div/div/div[2]/div/a/div/div[2]/ul/li[1]/span[2]') 
        bathroom_list = driver.find_elements(By.XPATH, '//div[1]/div/div/div/div[2]/div/a/div/div[2]/ul/li[2]/span[2]') 
        price_list =driver.find_elements(By.XPATH, '//div[1]/div/div/div/div[2]/div/a/div/div[1]/div/p') 
        desc_list = driver.find_elements(By.XPATH, '//div[1]/div/div/div/div[2]/div/a/div/div[3]/p') 
        date_added_list = driver.find_elements(By.XPATH, '//div[1]/div/div/div/div[2]/div/a/div/ul[2]/li') 
        agent_list_list = driver.find_elements(By.XPATH, '//div[1]/div/div/div/div[2]/div/a/div/div[1]/div/p')
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(address_item.text + '. ' + type_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)


        time.sleep(1.3)
        # get the height of the page
        page_height = driver.execute_script("return document.body.scrollHeight;")

        # scroll to the middle of the page using JavaScript
        driver.execute_script(f"window.scrollTo(0, {page_height//1.37});")

        # Find the next button
        
        try:
            next_botton = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/main/div/div[4]/div[2]/section/div[2]/div[4]/nav/div[3]/div/a')

            # Find the last next button 
            last_botton_check = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/main/div/div[4]/div[2]/section/div[2]/div[4]/nav/div[3]/div/a')
        except:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
            
        #Click the next button if it's not the last 
        time.sleep(5.5)
        #driver.find_element(By.XPATH, '/html/body/div[6]/div[3]/div[3]/div/div/div[4]/div/div[2]/div/a/button')
        next_botton.click()
        
        #Increment the page number
        i += 1
        time.sleep(2)
                
        
    # Create a dataframe to store data scrapped for each postcode    
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'Zoopla', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}S{i+1:05d}ZP' for i in range(len(df1))])
    print('Total numbers of properties available in ' + postcode + ' is ' + str(df1.shape[0]))

     # Concat the dataframe obtain for all postcodes
    df = pd.concat([df, df1], ignore_index=True)
    
     # Return a dataframe
    return df1



In [19]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = zoopla_sales(postcode, 'Sales', 'Zoopla', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))



I no see anything bros
scraping page 1 from BR1
------------------------------- SCRAPING COMPLETED FOR BR1
Total numbers of properties available in BR1 is 0
I no see anything bros
scraping page 1 from BR2
------------------------------- SCRAPING COMPLETED FOR BR2
Total numbers of properties available in BR2 is 0
I no see anything bros
scraping page 1 from BR3
------------------------------- SCRAPING COMPLETED FOR BR3
Total numbers of properties available in BR3 is 0



KeyboardInterrupt

